In [1]:
from gurobipy import *
# from math import *
# import math
import numpy as np
import pandas as pd
import timeit
from xlwt import Workbook

In [3]:
data1 = pd.read_excel('Group_6.xlsx', sheet_name='Flight')
data2 = pd.read_excel('Group_6.xlsx', sheet_name='Itinerary')
data3 = pd.read_excel('Group_6.xlsx', sheet_name='Recapture Rate')
data4 = pd.read_excel('Group_6.xlsx', sheet_name='Aircraft')


In [4]:
data1

,Flight Number,ORG,DEST,Departure,Arrival,A330,A340,B737,B738
0,AR1132,LIMC,KJFK,23:55:00,11:10:00,19797.633985,24304.697082,1000000.000000,1000000.000000
1,AR1133,KJFK,LIMC,16:10:00,04:20:00,20186.704183,24878.728342,1000000.000000,1000000.000000
2,AR1140,LDZA,LTFM,20:05:00,20:50:00,11225.286689,12235.857049,8753.575914,9021.988449
3,AR1160,LIMC,ZBAA,22:55:00,10:35:00,19976.944094,24569.026366,1000000.000000,1000000.000000
4,AR1161,ZBAA,LIMC,14:20:00,03:00:00,20390.933229,25180.760278,1000000.000000,1000000.000000
...,...,...,...,...,...,...,...,...,...
203,AR1893,BIKF,LIMC,19:05:00,21:31:00,13990.714954,15984.767576,11514.338562,11401.743746
204,AR1910,LIMC,SBGL,21:30:00,01:20:00,15402.914350,17959.826122,13038.003598,12685.016269
205,AR1911,SBGL,LIMC,02:40:00,06:55:00,15756.057036,18459.139309,13429.537738,13012.099800
206,AR1920,LIMC,MMMX,04:10:00,14:50:00,19539.195511,23924.402515,1000000.000000,1000000.000000


In [5]:
data2

,Itin No.,Origin,Destination,Demand,Fare,Stops,Leg 1,Leg 2
0,0,CYYZ,LIMC,16,108,0,AR1365,0
1,1,CYYZ,EDDB,16,54,1,AR1365,AR1673
2,2,CYYZ,ZSPD,33,48,1,AR1365,AR1260
3,3,CYVR,LIMC,20,107,0,AR1255,0
4,4,CYVR,EDDB,20,51,1,AR1255,AR1673
...,...,...,...,...,...,...,...,...
551,551,LIMC,ZBAA,60,109,0,AR1160,0
552,552,LIMC,OMDB,67,105,0,AR1302,0
553,553,LIMC,EDDF,30,53,0,AR1304,0
554,554,LIMC,EDDB,16,53,0,AR1667,0


In [6]:
data3

,From Itinerary,To Itinerary,Recapture Rate
0,54,58,0.39
1,56,59,0.35
2,58,54,0.35
3,59,56,0.34
4,63,66,0.38
...,...,...,...
163,513,418,0.15
164,515,464,0.18
165,516,444,0.19
166,523,451,0.11


In [7]:
data4

,Type,Units,Seats,TAT
0,A330,4,210,50
1,A340,2,220,60
2,B737,8,120,35
3,B738,29,140,35


In [8]:
airports = ['LIMC',
'KJFK',
'LDZA',
'ZBAA',
'FACT',
'CYVR',
'ZSPD',
'OMDB',
'EDDF',
'VIDP',
'SAEZ',
'CYYZ',
'EBBR',
'LTFM',
'EETN',
'LFPG',
'LPPT',
'EIDW',
'RJAA',
'LKPR',
'LIMF',
'ESSA',
'EDDB',
'EHEH',
'LFQQ',
'EFHK',
'LEMD',
'BIKF',
'EDDM',
'EFRO',
'LGAV',
'SBGL',
'MMMX']

flight_no = data1['Flight Number'].tolist()    # Flights list
origin_airport = data1['ORG'].tolist()
destination_airport = data1['DEST'].tolist()

itinerary = data2['Itin No.'].tolist()
demand = data2['Demand'].tolist()       # Daily unconstrained demand
fare = data2['Fare'].tolist()
leg1 = data2['Leg 1'].tolist()
leg2 = data2['Leg 2'].tolist()
stops = data2['Stops'].tolist()    # Ground arcs ?

from_itinerary = data3['From Itinerary'].tolist()
to_itinerary = data3['To Itinerary'].tolist()
recapture_rate = data3['Recapture Rate'].tolist()       # Recapture rate of pax

ac_type = data4['Type'].tolist()     # Type of aircraft: A330, A340, B737, B738
seats = data4['Seats'].tolist()
AC_units = data4['Units'].tolist()     # No of AC for each type

fare_fictious = 0
b_p_r_fictious = 1


In [9]:
# inbound = []
# for s in range(len(stops)):
#     if stops[s] == 1:
#         inbound.append(leg1[s])

# outbound1 = []
# for s in range(len(stops)):
#     if stops[s] == 0:
#         outbound1.append(leg1[s])

# outbound2 = []
# for s in range(len(stops)):
#     if stops[s] == 1:
#         outbound2.append(leg2[s])

# outbound = outbound1 + outbound2
 

In [10]:
outbound = []
for o in range(len(origin_airport)):
    # for k in range(len(ac_type)):
    outbound.append(leg1[o])

inbound = []
for o in range(len(destination_airport)):
    # for k in range(len(ac_type)):
    inbound.append(leg2[o])



In [11]:
range(len(outbound))

range(0, 208)

In [12]:
inbound

[0,
 'AR1673',
 'AR1260',
 0,
 'AR1673',
 'AR1260',
 0,
 'AR1832',
 'AR1471',
 'AR1292',
 'AR1822',
 'AR1240',
 'AR1651',
 0,
 'AR1673',
 'AR1260',
 0,
 'AR1406',
 'AR1471',
 'AR1292',
 'AR1822',
 'AR1240',
 'AR1507',
 'AR1651',
 'AR1600',
 'AR1737',
 0,
 'AR1691',
 'AR1722',
 'AR1376',
 'AR1322',
 'AR1852',
 0,
 'AR1832',
 'AR1406',
 'AR1471',
 'AR1292',
 'AR1822',
 'AR1240',
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 'AR1412',
 'AR1446',
 'AR1671',
 'AR1454',
 'AR1816',
 0,
 'AR1651',
 'AR1737',
 'AR1402',
 'AR1830',
 'AR1653',
 'AR1412',
 'AR1446',
 0,
 0,
 'AR1402',
 'AR1830',
 'AR1501',
 'AR1412',
 'AR1446',
 'AR1671',
 0,
 0,
 'AR1501',
 'AR1653',
 'AR1412',
 'AR1671',
 0,
 0,
 0,
 0,
 0,
 'AR1653',
 'AR1412',
 'AR1446',
 'AR1671',
 'AR1454',
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 'AR1454',
 'AR1473',
 'AR1647',
 'AR1252',
 0,
 0,
 'AR1473',
 0,
 0,
 0,
 0,
 0,
 0,
 'AR1454',
 'AR1816',
 'AR1473',
 'AR1647',
 'AR1252',
 'AR1684',
 0,
 'AR1816',
 'AR1473',
 'AR1647',
 'AR1252',
 '

In [13]:
delta_leg1 = np.zeros([len(flight_no), len(itinerary)])
for i in range(len(flight_no)):
    for p in range(len(itinerary)):
        if flight_no[i] == leg1[p]: 
            delta_leg1[i, p] = 1

delta_leg2 = np.zeros([len(flight_no), len(itinerary)])
for i in range(len(flight_no)):
    for p in range(len(itinerary)):
        if flight_no[i] == leg2[p]: 
            delta_leg2[i, p] = 1        

delta = delta_leg1 + delta_leg2

# csv_file_path = 'delta.csv'

# # Save 'delta' array to a CSV file
# np.savetxt(csv_file_path, delta, delimiter=',', fmt='%d')

In [14]:
delta_leg1

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [15]:
delta_leg2

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [16]:
# costs = {}
# for i, flight in enumerate(flight_no):
#     for k , aircraft in enumerate(ac_type):
#         costs[i][k] = [flight][aircraft]

In [17]:
# costs = {}
# for i, flight in enumerate(flight_no):
#     costs[i] = {}
#     for k, aircraft in enumerate(ac_type):
#         costs[i][aircraft] = data1.at[i, aircraft]  # Replace your_data_frame with your DataFrame name

In [18]:
# # Define the costs matrix
# costs = []
# for i in range(len(flight_no)):
#     costs.append([data1.at[i, ac] for ac in ac_type])


# # Displaying the costs matrix
# for row in costs:
#     print(row)

In [19]:
costs = [[data1.at[i, ac] for ac in ac_type] for i in range(len(flight_no))]


In [20]:
# # Get unique flight legs and convert them to strings conditionally
# flight_legs = set(map(str, data2['Leg 1'])).union({str(x) for x in data2['Leg 2'] if not isinstance(x, int)})

# # Create a matrix (DataFrame) to display 1 if a flight leg is part of the itinerary and 0 otherwise
# delta = pd.DataFrame(
#     np.zeros((len(data2), len(flight_legs))),
#     columns=sorted(flight_legs),
#     index=data2['Itin No.']
# )

# for index, row in data2.iterrows():
#     if not isinstance(row['Leg 2'], int):
#         matrix.loc[row['Itin No.'], str(row['Leg 2'])] = 1
#     matrix.loc[row['Itin No.'], str(row['Leg 1'])] = 1

# print(matrix)

In [21]:
print(costs)

[[19797.63398491833, 24304.69708224594, 1000000.0, 1000000.0], [20186.70418251475, 24878.72834226834, 1000000.0, 1000000.0], [11225.28668868037, 12235.8570492468, 8753.575914428899, 9021.9884493102], [19976.94409424513, 24569.02636573759, 1000000.0, 1000000.0], [20390.9332290603, 25180.76027820611, 1000000.0, 1000000.0], [13769.20743966, 15678.33337062094, 11281.78538034747, 11204.26644645922], [13971.03580454209, 15957.50464624136, 11493.60456414206, 11384.15579240081], [13769.20743966, 15678.33337062094, 11281.78538034747, 11204.26644645922], [13971.03580454209, 15957.50464624136, 11493.60456414206, 11384.15579240081], [13971.03580454209, 15957.50464624136, 11493.60456414206, 11384.15579240081], [14438.84009394555, 16607.5832142374, 11990.29097595565, 11804.50590386202], [13971.03580454209, 15957.50464624136, 11493.60456414206, 11384.15579240081], [14349.04677402738, 16482.48390191806, 11894.34371745166, 11723.45996697625], [13971.03580454209, 15957.50464624136, 11493.60456414206, 11

In [22]:
# a330_costs  = data1['A330'].tolist()
# a340_costs  = data1['A340'].tolist()
# b737_costs  = data1['B737'].tolist()
# b738_costs  = data1['B738'].tolist()

# keys = ['A330', 'A340', 'B737', 'B738']
# values = [a330_costs, a340_costs, b737_costs, b738_costs]
# op_costs = dict(zip(keys, values))p
# print(op_costs)

In [28]:
Q = np.zeros(len(flight_no))
for i in range(len(flight_no)):
    Q[i] = sum(delta[i, p] * demand[p] for p in range(len(itinerary)))

In [24]:
Q

array([ 73., 263.,  15.,  88.,  71., 242., 345., 478., 261., 200., 329.,
        91.,  72., 257., 114., 210., 175., 157., 162., 112., 241.,  99.,
       259.,  40., 112., 159., 100.,  56.,  76.,  26.,  27., 107.,  65.,
        72.,  60.,  21.,  97., 115.,  55., 140., 202.,  35., 153.,  96.,
       162., 111., 108., 137.,  61.,  73., 148., 108.,  27., 300., 232.,
       130., 125., 163., 185., 144., 190., 185., 215., 236., 202., 146.,
       331., 204., 324., 355., 158., 328., 351.,  57.,  52.,  31.,  61.,
        39.,  48.,  25.,  20.,  51.,  36.,  22.,  19.,  20.,  53.,  65.,
        26.,  11.,  52.,  69.,  13.,  16.,  26.,  40.,  67.,  55.,  64.,
        68.,  43., 102., 150., 105., 116., 129., 103., 119.,  88., 159.,
        34.,  21.,  20.,  22.,  69.,  55., 133., 126.,  77., 122.,  86.,
       109.,  39.,  87., 117.,  86.,  27., 102.,  82., 114., 125.,  30.,
        49.,  67.,  51.,   9.,  18.,  12.,   9.,  25.,  14.,  11.,  25.,
        65.,  78.,  87., 149., 104., 125.,  96.,  3

## Optimization model

In [25]:
m = Model('model')

# Decision variables
f = {} # direct pax flow from airport i to airport j
t = {} # number of px that would like to travel on itinerary p and are rellocated by the airline to itinerary r 
y = {} # number of aircraft of type k on the ground/flight arc a/n (integer)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2447081
Academic license 2447081 - for non-commercial use only - registered to yf___@tudelft.net


In [26]:
L = range(len(flight_no))
P = range(len(itinerary))
N = range(len(airports))
K = range(len(ac_type))
I = range(len(inbound))
O = range(len(outbound))
N_plus = range(len(origin_airport))
N_minus = range(len(destination_airport))

In [30]:
A = [(i, k) for i in L for k in K]
B = [(p, r) for p in P for r in P]
C = [(i, k) for i in L for k in K]

f = m.addVars(L, K, vtype=GRB.BINARY, name="f")
y = m.addVars(G, K, vtype=GRB.INTEGER, name="y")
t = m.addVars(P, P, vtype=GRB.INTEGER, name="t")

objective = quicksum(c_ki[i][k] * f[i,k] for i in L for k in K) + \
            quicksum((fare_p[p] - b_rp[r][p] * fare_r[r]) * t[p,r] for p in P for r in P)
m.setObjective(objective, GRB.MINIMIZE)


m.update()
m.setObjective(m.getObjective(), GRB.MINIMIZE)  


# Flight Schedule Coverage
for i in L:
    m.addConstr(quicksum(f[i, k] for k in K) == 1)

# Balance at Each Node
for k in K:
    for n in N:
        m.addConstr(quicksum(f[i,k] for i in O[k,n]) - quicksum(f[i,k] for i in I[k,n]) == y[n,k] - y[n,k])

# Number of Aircraft Available
for k in K:
    m.addConstr(quicksum(y[a,k] + f[a,k] for a in NG[k]) <= AC[k])

# Seat Capacity and Demand
for i in L:
    m.addConstr(quicksum(SEATS[k] * f[i,k] for k in K) + quicksum(t[p,r] for p in P for r in P if delta[i][p][r]) >= Q[i])


m.update()

m.setParam('OutputFlag', 1)

m.optimize()

for v in m.getVars():
    if v.x > 0:
        print(f'{v.varName} = {v.x}')
# m.write("testout.sol")
status = m.status

TypeError: 'range' object is not callable

In [20]:
for c in m.getConstrs():
    print ('The dual value of %s : %g'%(c.constrName,c.pi))

AttributeError: Unable to retrieve attribute 'pi'

In [23]:
# Access solver logs to retrieve dual values
if m.status == GRB.OPTIMAL:
    # Retrieve solver logs
    solver_logs = m.getAttr('X', m.getVars())

    # Extract dual values from solver logs
    dual_values = {}
    for log in solver_logs:
        if 'RHS' in log:
            dual_values[log] = solver_logs[log]
    
    # Print or access the dual values for each constraint
    for constraint, dual in dual_values.items():
        print(f"Dual value for constraint {constraint}: {dual}")
else:
    print("Optimization was not successful. Check the status before retrieving dual values.")


TypeError: argument of type 'float' is not iterable